# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

### Table: Matches

* Match_ID (int): unique ID per match  
* Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)  
* Season (int): Season the match took place in (usually covering the period of August till May of the following year)  
* Date (str): Date of the match  
* HomeTeam (str): Name of the home team  
* AwayTeam (str): Name of the away team
* FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
* FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
* FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

### Table: Teams

* Season (str): Football season for which the data is valid
* TeamName (str): Name of the team the data concerns
* KaderHome (str): Number of Players in the squad
* AvgAgeHome (str): Average age of players
* ForeignPlayersHome (str): Number of foreign players (non-German, non-English respectively) playing for the team
* OverallMarketValueHome (str): Overall market value of the team pre-season in EUR (based on data from transfermarkt.de)
* AvgMarketValueHome (str): Average market value (per player) of the team pre-season in EUR (based on data from transfermarkt.de)
* StadiumCapacity (str): Maximum stadium capacity of the team's home stadium

### Table: Unique Teams

* TeamName (str): Name of a team
* Unique_Team_ID (int): Unique identifier for each team

### Table: Teams_in_Matches

*  Match_ID (int): Unique match ID
* Unique_Team_ID (int): Unique team ID (This table is used to easily retrieve each match a given team has played in)

## Outline
* Get only info for 2011
* Get weather info for all game days (berlin as location for all games)
* Calculate total number of goals and total number of wins
* Calculate win pct on rainy days (just means if it rained at all that day)
* Make a hist of wins and losses
* Store that hist as an IMG and put it in the database
* Load the data into MongoDB

In [67]:
import time
import datetime

In [74]:
import requests

In [2]:
import sqlite3
conn = sqlite3.Connection('database.sqlite')
c = conn.cursor()

In [3]:
import pandas as pd

In [6]:
c.execute('''select name from sqlite_master where type = 'table';''').fetchall()

[('sqlite_sequence',),
 ('Matches',),
 ('Teams_in_Matches',),
 ('Teams',),
 ('Unique_Teams',)]

In [27]:
tablesa = c.execute('''select name from sqlite_master where type = 'table' AND 
    name NOT LIKE 'sqlite_%';''').fetchall()
tablesa

[('Matches',), ('Teams_in_Matches',), ('Teams',), ('Unique_Teams',)]

In [28]:
tables = []
for table in tablesa:
    tables.append(table[0]) 

In [29]:
tables

['Matches', 'Teams_in_Matches', 'Teams', 'Unique_Teams']

In [37]:
i = 1
for table in tables:
    print (table)
    query = 'select * from {}'.format(table)
    c.execute(query)
    df = pd.DataFrame(c.fetchall())
    df.columns = [x[0] for x in c.description]
    break
    i += 1

Matches


In [38]:
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [40]:
c.execute('select * from Teams_in_Matches')
df2 = pd.DataFrame(c.fetchall())
df2.columns = [x[0] for x in c.description]
df2.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [41]:
c.execute('select * from Teams')
df3 = pd.DataFrame(c.fetchall())
df3.columns = [x[0] for x in c.description]
df3.head()

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


In [42]:
c.execute('select * from Unique_Teams')
df4 = pd.DataFrame(c.fetchall())
df4.columns = [x[0] for x in c.description]
df4.head()

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [70]:
matchesdf = df.loc[df['Season'] == 2011]
matchesdf.reset_index(inplace=True)
print (len(matchesdf))
matchesdf.head()

992


,index,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [88]:
matchesdf.Date

0      2012-03-31
1      2011-12-11
2      2011-08-13
3      2011-11-27
4      2012-02-18
5      2012-01-20
6      2012-02-04
7      2012-04-21
8      2011-09-18
9      2011-10-23
10     2011-10-01
11     2012-03-03
12     2011-08-27
13     2012-03-17
14     2011-11-06
15     2012-05-05
16     2012-04-11
17     2011-08-27
18     2011-12-17
19     2012-02-18
20     2012-02-03
21     2011-08-13
22     2011-10-29
23     2012-01-22
24     2011-12-03
25     2012-04-14
26     2011-09-18
27     2012-03-25
28     2012-03-10
29     2012-04-07
          ...    
962    2012-04-28
963    2012-04-28
964    2012-04-28
965    2012-04-29
966    2012-04-29
967    2012-04-30
968    2012-05-01
969    2012-05-01
970    2012-05-02
971    2012-05-02
972    2012-05-05
973    2012-05-06
974    2012-05-06
975    2012-05-06
976    2012-05-06
977    2012-05-06
978    2012-05-06
979    2012-05-06
980    2012-05-07
981    2012-05-08
982    2012-05-13
983    2012-05-13
984    2012-05-13
985    2012-05-13
986    201

In [99]:
s = matchesdf.Date[981]
s

'2012-05-08'

In [100]:
time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple())

1336449600.0

In [102]:
response = requests.get('https://api.darksky.net/forecast/b9523226a8bb03c346fdfe0c4cd8b86a/52.514456, 13.398062,1336449600?exclude=currently,minutely,hourly,flags')

In [103]:
response.status_code

200

In [104]:
print(response.text)

{"latitude":52.514456,"longitude":13.398062,"timezone":"Europe/Berlin","daily":{"data":[{"time":1336428000,"summary":"Mostly cloudy until evening.","icon":"partly-cloudy-day","sunriseTime":1336447340,"sunsetTime":1336502762,"moonPhase":0.59,"precipIntensity":0,"precipIntensityMax":0,"precipProbability":0,"temperatureHigh":67.54,"temperatureHighTime":1336492800,"temperatureLow":52.01,"temperatureLowTime":1336532400,"apparentTemperatureHigh":67.54,"apparentTemperatureHighTime":1336492800,"apparentTemperatureLow":52.01,"apparentTemperatureLowTime":1336532400,"dewPoint":45.12,"humidity":0.65,"windSpeed":5.98,"windGust":10.99,"windGustTime":1336492800,"windBearing":131,"cloudCover":0.62,"uvIndex":5,"uvIndexTime":1336467600,"visibility":6.21,"temperatureMin":48.23,"temperatureMinTime":1336442400,"temperatureMax":67.54,"temperatureMaxTime":1336492800,"apparentTemperatureMin":47.23,"apparentTemperatureMinTime":1336453200,"apparentTemperatureMax":67.54,"apparentTemperatureMaxTime":1336492800}]}

In [ ]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    pass

In [ ]:
class MongoHandler():
    pass

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!